In [20]:
# Attention is All You Need paper: https://arxiv.org/abs/1706.03762
# OpenAI GPT-3 paper: https://arxiv.org/abs/2005.14165 
# OpenAI ChatGPT blog post: https://openai.com/blog/chatgpt/

In [21]:
# open the file data/input.txt
with open('data/input.txt', 'r') as file:
    text = file.read()

In [22]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you


In [23]:
# all the unique characters that appear in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print('all the unique characters that appear in the text:', chars)
print('vocab_size:', vocab_size)

all the unique characters that appear in the text: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
vocab_size: 65


In [24]:
# character level tokenization
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
# encode and decode
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# encode the text
test_string = 'hello world'
encoded = encode(test_string)
print(encoded)
# decode the text
decoded = decode(encoded)
print(decoded)

[46, 43, 50, 50, 53, 1, 61, 53, 56, 50, 42]
hello world


In [25]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])


torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [26]:
# split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest will be validation
train_data = data[:n]
val_data = data[n:]

In [27]:
block_size = 8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

## x = torch.stack([data[i:i+block_size] for i in ix]) 
## above is equivalent to the following:
## result = []
## for i in ix:
##   result.append(data[i:i+block_size])

# radint is generateing random integers between 0 and len(data) - block_size (how many integers to generate)
def get_batch(split): 
    # generate a small batch of data of inputs x and targets y.
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    # y also need [batch_size] chars since its predicting the next character in each sequence
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('-'*100)

for b in range(batch_size):
   for t in range(block_size):
      context = xb[b, :t+1]
      target = yb[b, t]
      print(f"when input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----------------------------------------------------------------------------------------------------
when input is [24] the target is 43
when input is [24, 43] the target is 58
when input is [24, 43, 58] the target is 5
when input is [24, 43, 58, 5] the target is 57
when input is [24, 43, 58, 5, 57] the target is 1
when input is [24, 43, 58, 5, 57, 1] the target is 46
when input is [24, 43, 58, 5, 57, 1, 46] the target is 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target is 39
when input is [44] the target is 53
when input is [44, 53] the target is 56
when input is [44, 53, 56] the target is

In [41]:
# TODO: get to know more about Byram language model.
import torch 
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx: (B, T)
        # logits: (B, T, C)
        logits = self.token_embedding_table(idx)
        
        if targets is not None:
            # 训练模式：需要计算 loss
            B, T, C = logits.shape
            logits_reshaped = logits.view(B*T, C)
            targets_reshaped = targets.view(B*T)
            loss = F.cross_entropy(logits_reshaped, targets_reshaped)
        else:
            # 生成模式：不需要 loss
            loss = None
        
        # 返回的 logits 保持 3维 (B, T, C)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx: (B, T)
        for _ in range(max_new_tokens):
            # 前向传播，不传 targets，所以不计算 loss
            logits, loss = self(idx)
            # logits 现在是 (B, T, C)
            # 只取最后一个时间步的预测
            logits = logits[:, -1, :]  # (B, C)
            # 转成概率
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # 采样下一个 token
            # idx_next = torch.argmax(probs, dim=-1, keepdim=True)  # (B, 1) # this will always choose the most likely token   
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1) # this will choose a random token based on the probability distribution
            # 拼接到序列后面
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)
        return idx
    

model = BigramLanguageModel(vocab_size)
out = model(xb)
print(out)

(tensor([[[ 1.4303,  0.4037,  1.9524,  ...,  0.2820, -1.0089,  0.6827],
         [-0.5951,  0.1419,  1.4275,  ...,  0.6173, -0.5437,  1.1774],
         [ 0.3034,  0.2829, -0.2321,  ...,  0.8869, -1.9152, -0.4107],
         ...,
         [-0.3887, -1.3260,  0.9119,  ..., -1.4301,  0.6435, -0.3685],
         [ 0.3595,  1.6837,  0.5770,  ..., -1.8711, -0.1925,  0.8304],
         [-0.2734, -0.5149,  0.4448,  ..., -0.9727,  0.2400, -0.6197]],

        [[ 2.0527, -1.9720, -0.2829,  ...,  0.4928,  1.5594,  0.3016],
         [-0.0668,  0.2809, -0.6740,  ..., -1.0420,  0.1128, -0.6462],
         [-2.8751, -0.5480, -1.8130,  ..., -1.7825, -0.7748,  0.6598],
         ...,
         [ 0.3595,  1.6837,  0.5770,  ..., -1.8711, -0.1925,  0.8304],
         [ 2.0527, -1.9720, -0.2829,  ...,  0.4928,  1.5594,  0.3016],
         [ 0.1759,  0.2595,  1.1936,  ...,  0.1249, -0.5591,  0.5565]],

        [[ 0.0419,  0.2686,  0.0077,  ..., -0.3085,  0.1822,  0.5654],
         [ 0.1759,  0.2595,  1.1936,  ...,  

In [42]:
#
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


rTQsZzNs,gLwWKQcSaj:TIjLUU$DWSXDRDrL'lhiEt'BlNyI&UjlyeL
xAKccxzo&:?EEsDGg!x&Ticyw B PgE.wkmxH:T'.vXc


In [43]:
# create  a simple pytorch optimizer 
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [44]:
batch_size = 32
for steps in range(10000): 
    xb, yb = get_batch('train')
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if steps % 100 == 0:
        print(f"step {steps}: loss {loss.item()}")

print(loss.item())

step 0: loss 4.802587032318115
step 100: loss 4.597634315490723
step 200: loss 4.526169300079346
step 300: loss 4.453063488006592
step 400: loss 4.321571350097656
step 500: loss 4.108897686004639
step 600: loss 4.071845054626465
step 700: loss 4.058953285217285
step 800: loss 3.8918159008026123
step 900: loss 3.7922182083129883
step 1000: loss 3.772552728652954
step 1100: loss 3.682431936264038
step 1200: loss 3.589136838912964
step 1300: loss 3.5480942726135254
step 1400: loss 3.468914270401001
step 1500: loss 3.3877973556518555
step 1600: loss 3.3869128227233887
step 1700: loss 3.2994039058685303
step 1800: loss 3.247647285461426
step 1900: loss 3.1561388969421387
step 2000: loss 3.1826529502868652
step 2100: loss 3.094543933868408
step 2200: loss 3.0225722789764404
step 2300: loss 2.9976327419281006
step 2400: loss 3.021346092224121
step 2500: loss 2.935741901397705
step 2600: loss 2.877305507659912
step 2700: loss 2.8656115531921387
step 2800: loss 2.879413604736328
step 2900: loss

In [45]:
#
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=600)[0].tolist()))


th m, nghe y IAn we.
Oy hing be!
F on.
Smalllousthisthe eloor gs ag.
Sce,
We:
S dy ad;

ZAR:
Anerby d m! lee!
Touginaroufe.

ERYar teath, re tld I?
V:
Theend he thathidr fyovere wnd e olen RI urk ioo,

A at, meas b fore te, at lowd t-fouly merdalous,
OLame fod wat ong:
So,

BRG u, me ted whont he fotoomo thind acrthyTI ithahenomsth toss h wayopf e f phile hilimefise. titile?
Cipese

UTISethaprss,
AMERG af ourtw,
NGS:
Whee the IThen:
N:
NGBOfo,
THNNCELE caust hush sudo'dprlik
VINTESToul-pofalys nd adipouro:
IXENTCat'
Ant nt t is youg le thearo qud Then
e crgoll?
STh r--hepen thisheer-g s fanodo


In [ ]:
# we could see even this simple model could start to generate text. 
# and so far the tokens are not talked to each other. 